# Lab in Data Science: Final Project

Pierre Fouche, Matthias Leroy and Raphaël Steinmann

## Imports

In [1]:
%matplotlib inline
import matplotlib.pylab as plt
import numpy as np
plt.rcParams['figure.figsize'] = (10,6)
plt.rcParams['font.size'] = 18
plt.style.use('fivethirtyeight')

In [36]:
import getpass
import pyspark
from datetime import datetime,timedelta
from pyspark.sql import SparkSession
import pyspark.sql.functions as functions
from pyspark.sql.types import BooleanType
from pyspark.sql.window import Window
import math
import helpers
import pickle

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Initialize the `SparkSession`

In [3]:
conf = pyspark.conf.SparkConf()
conf.setMaster('yarn')
conf.setAppName('project-{0}'.format(getpass.getuser()))
conf.set('spark.executor.memory', '6g')
conf.set('spark.executor.instances', '6')
conf.set('spark.port.maxRetries', '100')
sc = pyspark.SparkContext.getOrCreate(conf)
conf = sc.getConf()
sc

<SparkContext master=yarn appName=project-rbsteinm>

In [4]:
# init spark session
spark = SparkSession(sc)

## Data Processing

### Cleaning metadata
First, let's clean the metadata dataframe. We will use the SBB data limited around the Zurich area. We will focus on all the stops within 10km of the Zurich train station. Let's get rid of all the stations that are too far away from Zurich:

In [5]:
RELOAD_METADATA = True
if RELOAD_METADATA:
    # load metadata
    raw_metadata = spark.read.load('/datasets/project/metadata', format='com.databricks.spark.csv', header='false', sep='\\t')

    # remove multiple spaces
    metadata = raw_metadata.withColumn('_c0', functions.regexp_replace(raw_metadata._c0, '\s+', ' '))
    # split into columns
    metadata = metadata.withColumn('name', functions.split(metadata._c0, '%')[1])
    for (name, index, type_) in [('station_ID',0, 'int'), ('long',1, 'double'), ('lat',2, 'double'), ('height',3, 'int')]:
        metadata = metadata.withColumn(name, functions.split(metadata._c0, ' ')[index].cast(type_))
    # remove useless column
    metadata = metadata.drop('_c0')
    # trim name column to remove left/right blank
    metadata = metadata.withColumn('name', functions.trim(metadata.name))

    # coordinates of Zürich main train station
    lat_zurich = 47.3782
    long_zurich = 8.5402

    # convert to pandas dataframe
    pandas_df = metadata.toPandas()
    
    # keep only the stops that are located < 10km from Zurich HB
    pandas_df['distance_to_zh'] = pandas_df.apply(lambda x: helpers.distance(x['long'], x['lat'], long_zurich, lat_zurich), axis=1)
    pandas_df = pandas_df[pandas_df['distance_to_zh'] < 10]

    # recreate spark dataframe from pandas dataframe
    metadata = spark.createDataFrame(pandas_df)
    # create dict of stations from pandas dataframe
    stations = pandas_df.set_index('station_ID').to_dict('index')

    # dump metadata in pickle
    with open('./data/metadata.pickle', 'wb') as handle:
        pickle.dump(stations, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
# load metadata from pickle
with open('./data/metadata.pickle', 'rb') as handle:
    stations = pickle.load(handle)

### Cleaning main dataset

In [12]:
# load full data
# raw_df = spark.read.load('/datasets/project/istdaten/*/*', format='csv', header='true', inferSchema='true', sep=';')
# load sample data
raw_df = spark.read.load('/datasets/project/istdaten/2018/01', format='csv', header='true', inferSchema='true', sep=';')

In [13]:
# rename the fields german -> english
fields = {
    'BETRIEBSTAG':'date',
    'FAHRT_BEZEICHNER':'trip_id',
    'PRODUKT_ID':'transport_type',
    'LINIEN_ID':'train_id',
    'LINIEN_TEXT':'line',
    'VERKEHRSMITTEL_TEXT':'train_type',
    'ZUSATZFAHRT_TF':'additional_trip',
    'FAELLT_AUS_TF':'trip_failed',
    'HALTESTELLEN_NAME':'stop_name',
    'BPUIC':'stop_id',
    'ANKUNFTSZEIT':'schedule_arrival',
    'AN_PROGNOSE':'real_arrival',
    'AN_PROGNOSE_STATUS':'arr_forecast_status',
    'ABFAHRTSZEIT':'schedule_dep',
    'AB_PROGNOSE':'real_dep',
    'AB_PROGNOSE_STATUS':'dep_forecast_status',
    'DURCHFAHRT_TF':'no_stop_here'
}

df = raw_df.selectExpr([k + ' as ' + fields[k] for k in fields])

In [14]:
# refactor dates
df = df.withColumn('date', functions.from_unixtime(functions.unix_timestamp('date', 'dd.MM.yyyy')))
df = df.withColumn('schedule_arrival', functions.from_unixtime(functions.unix_timestamp('schedule_arrival', 'dd.MM.yyyy HH:mm')))
df = df.withColumn('real_arrival', functions.from_unixtime(functions.unix_timestamp('real_arrival', 'dd.MM.yyyy HH:mm')))
df = df.withColumn('schedule_dep', functions.from_unixtime(functions.unix_timestamp('schedule_dep', 'dd.MM.yyyy HH:mm')))
df = df.withColumn('real_dep', functions.from_unixtime(functions.unix_timestamp('real_dep', 'dd.MM.yyyy HH:mm')))

In [15]:
# add a column containing the weekday (monday=1, sunday=6)
df = df.withColumn('weekday', helpers.get_weekday(df.date))

In [16]:
# keep only the rows with stops near zurich
df = df.where(df.stop_id.isin([int(x) for x in list(pandas_df.station_ID.unique())]))

In [17]:
# there is still 51'571'541 rows in zurich area
# df.count()

In [18]:
# keep only date after the 10th of december, because the schedule changed
df = df.where(df.date > '2017-12-10 00:00:00')

In [19]:
# discard the rows when there is no stop here
df2 = df.where(df.no_stop_here == 'false')

In [20]:
# discard ill-formated rows where the train leaves a station before arriving in it
df2 = df2.where((df2.schedule_dep >= df2.schedule_arrival) | functions.col('schedule_arrival').isNull() | functions.col('schedule_dep').isNull())

## Modeling the network

### From stops to trips

In [21]:
# create a column with the schedule time that will be used to build the network
df2 = df2.withColumn('schedule_time', helpers.date_choice(df2.schedule_arrival, df2.schedule_dep))
#df2 = df2.withColumn('schedule_time', functions.from_unixtime(functions.unix_timestamp('schedule_time', 'dd.MM.yyyy HH:mm')))

# create a column that tells if a stop is the first/last one of its trip or in the middle
df2 = df2.withColumn('stop_type', helpers.stop_type(df2.schedule_dep, df2.schedule_arrival))

In [22]:
trips = df2.select(['trip_id', 'date', 'schedule_time', 'stop_id', 'stop_type', 'schedule_arrival', 'schedule_dep', 'line']).orderBy(['trip_id', 'schedule_time'], ascending=[0,0])

In [23]:
# duplicate the dataframe, shift the copy of one row and append it to the original
# this way, we have for each row the current stop and the next stop
w = Window().partitionBy(functions.col('trip_id')).orderBy(functions.col('trip_id'))
trips2 = trips.select("*", functions.lag("trip_id").over(w).alias("next_tid"))
trips2 = trips2.select("*", functions.lag("schedule_time").over(w).alias("next_time"))
trips2 = trips2.select("*", functions.lag("stop_id").over(w).alias("next_sid"))
trips2 = trips2.select("*", functions.lag("stop_type").over(w).alias("next_type"))
trips2 = trips2.select("*", functions.lag("schedule_arrival").over(w).alias("next_sched_arr"))
trips2 = trips2.select("*", functions.lag("schedule_dep").over(w).alias("next_sched_dep"))

trips2 = trips2.where(trips2.next_time.isNotNull())

In [25]:
# create a new column telling if the edge is valid or not
# (i.e. if the stop and next stop are really part of the same ride)
trips3 = trips2.withColumn('is_valid', helpers.edge_is_valid(trips2.trip_id, trips2.schedule_time, trips2.stop_id, trips2.stop_type, trips2.next_tid, trips2.next_time, trips2.next_sid, trips2.next_type, trips2.schedule_dep,trips2.next_sched_arr))

In [26]:
# keep only valid edges
trips4 = trips3.filter(trips3.is_valid=='true')

### For each day of the week, model the network
Get the edges of the network and the departure/arrival times for each trip (edge=trip)
We assume the schedule repeat every week, and we generate one schedule per weekday.
Days off have the same schedules as sundays.

In [28]:
# creating a model for each day of the week
# this code needs to be run only once
typical_monday = '2018-01-15 00:00:00'
typical_tuesday = '2018-01-16 00:00:00'
typical_wednesday = '2018-01-17 00:00:00'
typical_thursday = '2018-01-18 00:00:00'
typical_friday = '2018-01-19 00:00:00'
typical_saturday = '2018-01-20 00:00:00'
typical_sunday = '2018-01-21 00:00:00'
typical_week = [typical_monday,typical_tuesday,typical_wednesday,typical_thursday,typical_friday,typical_saturday,typical_sunday]

In [33]:
regenerate_models = False
days_names = ['monday','tuesday','wednesday','thursday','friday','saturday','sunday']

# generate one network for each weekday and store them in pickles
if regenerate_models:
    for (date, day_name) in zip(typical_week, days_names):
        network = (helpers.model_network(trips4, date))
        with open('./data/'+day_name+'.pickle', 'wb') as handle:
            helpers.network_to_datetime(network) # works inplace
            pickle.dump(network, handle, protocol=pickle.HIGHEST_PROTOCOL)
        print(str(day_name) + ' done')

In [34]:
# load the networks from the pickles
models = []
for day in days_names:
    with open('./data/'+ day +'.pickle', 'rb') as handle:
        network = pickle.load(handle)
    models.append(network)
    print(day + ' loaded')

monday loaded
tuesday loaded
wednesday loaded
thursday loaded
friday loaded
saturday loaded
sunday loaded


### Compute walking network

In [42]:
# compute walking network
walking_network = helpers.compute_walking_network(stations)
print('walking network loaded')

walking network loaded


## Shortest path algorithm

In [58]:
# example
helpers.shortest_path(models, walking_network, stations,8576218, 8590727, datetime(2018, 1, 15, 14))

[(8576218,
  8576217,
  datetime.datetime(2018, 1, 15, 14, 3),
  datetime.datetime(2018, 1, 15, 14, 4),
  '85:849:104861-01912-1',
  '916'),
 (8576217,
  8576216,
  datetime.datetime(2018, 1, 15, 14, 4),
  datetime.datetime(2018, 1, 15, 14, 5),
  '85:849:104861-01912-1',
  '916'),
 (8576216,
  8576209,
  datetime.datetime(2018, 1, 15, 14, 5),
  datetime.datetime(2018, 1, 15, 14, 6),
  '85:849:104861-01912-1',
  '916'),
 (8576209,
  8576208,
  datetime.datetime(2018, 1, 15, 14, 6),
  datetime.datetime(2018, 1, 15, 14, 7),
  '85:849:104861-01912-1',
  '916'),
 (8576208,
  8576207,
  datetime.datetime(2018, 1, 15, 14, 7),
  datetime.datetime(2018, 1, 15, 14, 8),
  '85:849:104861-01912-1',
  '916'),
 (8576207,
  8576206,
  datetime.datetime(2018, 1, 15, 14, 9),
  datetime.datetime(2018, 1, 15, 14, 9),
  '85:849:104861-01912-1',
  '916'),
 (8576206,
  8576205,
  datetime.datetime(2018, 1, 15, 14, 10),
  datetime.datetime(2018, 1, 15, 14, 10),
  '85:849:104861-01912-1',
  '916'),
 (8576205,


In [55]:
(1,)+(2,3,4)

(1, 2, 3, 4)

### Some tests

In [ ]:
# select two stops at random and create a shortest path. Do that 100 times
import random
shortest_paths = []
for i in range(100):
    if i %10==0:
        print(i, '% finished')
    source = random.choice(list(stations.keys()))
    dest = random.choice(list(stations.keys()))
    shortest_paths.append(shortest_path(models, walking_network, stations, source, dest, datetime(2018, 1, 15, 14)))  

In [ ]:
times = []
for path in shortest_paths:
    dep = path[0][1]
    arr = path[-1][2]
    times.append(round((arr-dep).total_seconds()/60))

In [86]:
nhops = [len(path) for path in shortest_paths]

In [89]:
min(nhops)
nhops.index(min(nhops))

48

In [90]:
times.index(min(times))

48

In [ ]:
shortest_paths[23]

In [ ]:
s = (shortest_paths[23][0][0])
d = (shortest_paths[23][-1][0])